In [ ]:
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = "TRUE"
os.environ['DASHSCOPE_API_KEY'] = "xxxxxxxxxxx"
os.environ['MODELSCOPE_API_TOKEN'] = 'xxxxxxxxxxxx'

In [ ]:
prompts = []
outputs = []

# 这个是python的猴子补丁(Monkey Patching) ，这里用于hook住modelscope_agent的代码，以便我们可以查看模型的input和output
def fix_1():
    import os
    import random
    import traceback
    from http import HTTPStatus
    from typing import Union

    import dashscope
    import json
    from dashscope import Generation
    from modelscope_agent.agent_types import AgentType

    from modelscope_agent.llm.base import LLM
    from modelscope_agent.llm.utils import DEFAULT_MESSAGE, CustomOutputWrapper

    class my_DashScopeLLM(LLM):
        name = 'dashscope_llm'

        def __init__(self, cfg):
            super().__init__(cfg)
            self.model = self.cfg.get('model', 'modelscope-agent-llm-v1')
            self.model_id = self.model
            self.generate_cfg = self.cfg.get('generate_cfg', {})
            self.agent_type = self.cfg.get('agent_type', AgentType.DEFAULT)

        def generate(self,
                     llm_artifacts: Union[str, dict],
                     functions=[],
                     **kwargs):
            prompts.append(llm_artifacts)

            # TODO retry and handle message
            try:
                if self.agent_type == AgentType.Messages:
                    messages = llm_artifacts if len(
                        llm_artifacts) > 0 else DEFAULT_MESSAGE
                    self.generate_cfg['use_raw_prompt'] = False
                    response = dashscope.Generation.call(
                        model=self.model,
                        messages=messages,
                        # set the random seed, optional, default to 1234 if not set
                        seed=random.randint(1, 10000),
                        result_format=
                        'message',  # set the result to be "message" format.
                        stream=False,
                        **self.generate_cfg)
                    llm_result = CustomOutputWrapper.handle_message_chat_completion(
                        response)
                else:
                    response = Generation.call(
                        model=self.model,
                        prompt=llm_artifacts,
                        stream=False,
                        **self.generate_cfg)
                    llm_result = CustomOutputWrapper.handle_message_text_completion(
                        response)
                outputs.append(llm_result)
                return llm_result
            except Exception as e:
                error = traceback.format_exc()
                error_msg = f'LLM error with input {llm_artifacts} \n dashscope error: {str(e)} with traceback {error}'
                print(error_msg)
                raise RuntimeError(error)

            if self.agent_type == AgentType.MS_AGENT:
                # in the form of text
                idx = llm_result.find('<|endofthink|>')
                if idx != -1:
                    llm_result = llm_result[:idx + len('<|endofthink|>')]
                return llm_result
            elif self.agent_type == AgentType.Messages:
                # in the form of message
                return llm_result
            else:
                # in the form of text
                return llm_result

        def stream_generate(self,
                            llm_artifacts: Union[str, dict],
                            functions=[],
                            **kwargs):
            print('stream_generate')
            prompts.append(llm_artifacts)

            total_response = ''
            try:
                if self.agent_type == AgentType.Messages:
                    self.generate_cfg['use_raw_prompt'] = False
                    responses = Generation.call(
                        model=self.model,
                        messages=llm_artifacts,
                        stream=True,
                        result_format='message',
                        **self.generate_cfg)
                else:
                    responses = Generation.call(
                        model=self.model,
                        prompt=llm_artifacts,
                        stream=True,
                        **self.generate_cfg)
            except Exception as e:
                error = traceback.format_exc()
                error_msg = f'LLM error with input {llm_artifacts} \n dashscope error: {str(e)} with traceback {error}'
                print(error_msg)
                raise RuntimeError(error)

            for response in responses:
                if response.status_code == HTTPStatus.OK:
                    if self.agent_type == AgentType.Messages:
                        llm_result = CustomOutputWrapper.handle_message_chat_completion(
                            response)
                        frame_text = llm_result['content'][len(total_response):]
                    else:
                        llm_result = CustomOutputWrapper.handle_message_text_completion(
                            response)
                        frame_text = llm_result[len(total_response):]
                    yield frame_text

                    if self.agent_type == AgentType.Messages:
                        total_response = llm_result['content']
                    else:
                        total_response = llm_result
                else:
                    err_msg = 'Error Request id: %s, Code: %d, status: %s, message: %s' % (
                        response.request_id, response.status_code, response.code,
                        response.message)
                    print(err_msg)
                    raise RuntimeError(err_msg)

    from modelscope_agent.llm import dashscope_llm
    dashscope_llm.DashScopeLLM = my_DashScopeLLM

fix_1() # 如果不需要查看模型的input和output，可以不用执行这个cell

In [ ]:
from modelscope_agent.tools import Tool
from modelscope_agent.llm import LLMFactory
from modelscope_agent.prompt import MSPromptGenerator, MrklPromptGenerator
from modelscope_agent.output_parser import MsOutputParser, MRKLOutputParser
from modelscope.utils.config import Config
from modelscope_agent.agent import AgentExecutor

model_name = 'modelscope-agent'
model_cfg = {
    "qwen-max": {
        "type": "dashscope",
        "model": "qwen-max",
        "generate_cfg": {
            "use_raw_prompt": True,
            "top_p": 0.8,
            "debug": False
        }
    },
    "modelscope-agent": {
        "type": "dashscope",
        "model": "modelscope-agent-llm-v1",
        "generate_cfg": {
            "use_raw_prompt": True,
            "top_p": 0.8,
            "seed": 666,
            "debug": True
        }
    }
}

llm = LLMFactory.build_llm(model_name, model_cfg)

In [ ]:
# prompt_generator = MrklPromptGenerator(llm=llm)
# output_parser = MRKLOutputParser()

prompt_generator = MSPromptGenerator()
output_parser = MsOutputParser()

from modelscope_agent.tools import Tool

class AliyunRenewInstanceTool(Tool):
    description = '续费一台包年包月ECS实例'
    name = 'RenewInstance'
    parameters: list = [{
        'name': 'instance_id',
        'description': 'ECS实例ID',
        'required': True
    },
        {
            'name': 'period',
            'description': '续费时长以月为单位',
            'required': True
        }
    ]

    def _local_call(self, *args, **kwargs):
        instance_id = kwargs['instance_id']
        period = kwargs['period']
        return {'result': f'成功为{instance_id}续费，续费时长{period}月'}


additional_tool_list = {
    'RenewInstance': AliyunRenewInstanceTool()
}

agent = AgentExecutor(llm, additional_tool_list=additional_tool_list, tool_retrieval=False,
                      prompt_generator=prompt_generator, output_parser=output_parser)


In [ ]:
# 重置对话，清空对话历史
agent.reset()
available_tool_list = ['RenewInstance']
agent.set_available_tools(available_tool_list)
agent.run("请帮我续费一台ECS实例，实例id是：i-rj90a7e840y5cde，续费时长10个月", remote=False, print_info=True)

In [ ]:
print(prompts[0])

In [ ]:
print(prompts[1])

In [ ]:
print(outputs[0])

In [ ]:
print(outputs[1])

In [ ]:
import json
txt = """
{
   "api_name": "RenewInstance",
   "url": "https://ecs.aliyuncs.com/renewinstance",
   "parameters": {
      "instance_id": "i-rj90a7e840y5cde",
      "period": 10
   }
}
"""
data = json.loads(txt)

In [ ]:
data